In [1]:
# ✅ Run this block first in your notebook

import nest_asyncio
import asyncio
import websockets
import json

import ssl
import certifi

#ssl_context = ssl.create_default_context(cafile=certifi.where())

ssl_context = ssl._create_unverified_context()

import asyncio
import websockets
import threading
import json
import time
import requests
from collections import OrderedDict


In [2]:
class LocalOrderBook:
    def __init__(self):
        self.bids = OrderedDict()
        self.asks = OrderedDict()

    def update(self, updates, side):
        book = self.bids if side == "b" else self.asks
        for update in updates:
            price, volume = float(update[0]), float(update[1])
            if volume == 0:
                book.pop(price, None)
            else:
                book[price] = volume

        if side == "b":
            self.bids = OrderedDict(sorted(book.items(), reverse=True))
        else:
            self.asks = OrderedDict(sorted(book.items()))

    def patch_from_snapshot(self, snapshot_bids, snapshot_asks):
        self._patch_side(snapshot_bids, self.bids, side="b")
        self._patch_side(snapshot_asks, self.asks, side="a")

    def _patch_side(self, snapshot, book, side):
        if len(book) >= 100:
            return

        for update in snapshot:
            price, volume = float(update[0]), float(update[1])
            if price not in book and len(book) < 100:
                book[price] = volume

        # re-sort after patch
        if side == "b":
            self.bids = OrderedDict(sorted(book.items(), reverse=True))
        else:
            self.asks = OrderedDict(sorted(book.items()))

    def top_of_book(self):
        best_bid = next(iter(self.bids.items()), (None, None))
        best_ask = next(iter(self.asks.items()), (None, None))
        return best_bid, best_ask

    def needs_patch(self):
        return len(self.bids) < 100 or len(self.asks) < 100

In [3]:
class KrakenL2Client:
    def __init__(self, pairs=["XBT/USD"], depth=100):
        self.pairs = pairs
        self.depth = depth
        self.uri = "wss://ws.kraken.com"
        self.loop = None
        self.thread = None
        self.stop_event = threading.Event()
        self.order_books = {pair: LocalOrderBook() for pair in self.pairs}
        self.last_patch_time = time.time()

    async def connect(self):
        async with websockets.connect(self.uri, ssl=ssl_context) as websocket:
            await websocket.send(json.dumps({
                "event": "subscribe",
                "pair": self.pairs,
                "subscription": {"name": "book", "depth": self.depth}
            }))
            print(f"📡 Subscribed to: {', '.join(self.pairs)}")

            while not self.stop_event.is_set():
                try:
                    message = await websocket.recv()
                    self.handle_message(json.loads(message))
                    self.maybe_patch_books()
                except Exception as e:
                    print(f"⚠️ Error: {e}")
                    break

    def maybe_patch_books(self):
        if time.time() - self.last_patch_time < 5:
            return  # Don't patch too frequently

        for pair in self.pairs:
            book = self.order_books[pair]
            if book.needs_patch():
                print(f"🔧 Patching {pair}...")
                bids, asks = self.fetch_snapshot(pair)
                if bids and asks:
                    book.patch_from_snapshot(bids, asks)
        self.last_patch_time = time.time()

    def fetch_snapshot(self, pair):
        symbol = pair.replace("/", "")
        url = f"https://api.kraken.com/0/public/Depth?pair={symbol}&count=100"
        try:
            resp = requests.get(url, timeout=5)
            data = resp.json()
            key = list(data["result"].keys())[0]
            return data["result"][key]["bids"], data["result"][key]["asks"]
        except Exception as e:
            print(f"❌ Snapshot error: {e}")
            return None, None

    def handle_message(self, message):
        if isinstance(message, dict):
            if message.get("event") in ["heartbeat", "systemStatus", "subscriptionStatus"]:
                return

        if isinstance(message, list) and len(message) > 1:
            data = message[1]
            pair = message[-1]
            book = self.order_books.get(pair)
            if not book:
                return

            if 'a' in data:
                book.update(data['a'], 'a')
            if 'b' in data:
                book.update(data['b'], 'b')

            bid, ask = book.top_of_book()
            print(f"📈 {pair} Top of Book → Bid: {bid[0]} | Ask: {ask[0]}")

    def start(self):
        self.loop = asyncio.new_event_loop()
        asyncio.set_event_loop(self.loop)
        self.loop.run_until_complete(self.connect())

    def run_in_thread(self):
        self.thread = threading.Thread(target=self.start)
        self.thread.start()

    def stop(self):
        self.stop_event.set()
        if self.loop:
            self.loop.call_soon_threadsafe(self.loop.stop)
        if self.thread:
            self.thread.join()

In [4]:
# ---------------------- USAGE ---------------------- #
if __name__ == "__main__":
    client = KrakenL2Client(pairs=["XBT/USD", "ETH/USD"])
    client.run_in_thread()

    try:
        while True:
            time.sleep(1)  # Keep main thread alive
    except KeyboardInterrupt:
        print("🛑 Stopping client...")
        client.stop()


📡 Subscribed to: XBT/USD, ETH/USD
📈 ETH/USD Top of Book → Bid: None | Ask: None
📈 XBT/USD Top of Book → Bid: None | Ask: None
📈 ETH/USD Top of Book → Bid: 2378.09 | Ask: None
📈 ETH/USD Top of Book → Bid: 2378.09 | Ask: 2418.35
📈 ETH/USD Top of Book → Bid: 2390.77 | Ask: 2418.35
📈 ETH/USD Top of Book → Bid: 2390.77 | Ask: 2418.35
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2418.35
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2418.35
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2418.35
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2402.71
📈 XBT/USD Top of Book → Bid: None | Ask: 103963.7
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2402.71
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2391.64
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2391.64
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2391.64
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2391.64
📈 XBT/USD Top of Book → Bid: None | Ask: 103788.0
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2391.64
📈 ETH/USD Top of Book → Bid: 2390.64 | Ask: 2391.

Exception in thread Thread-3 (start):
Traceback (most recent call last):
  File "C:\Users\trrallele\AppData\Local\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\trrallele\Momentum Metropolitan\REALEARN\CRYPTO STRATEGIES IN PYTHON\myenv2.0\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\trrallele\AppData\Local\anaconda3\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\trrallele\AppData\Local\Temp\ipykernel_19380\285807664.py", line 78, in start
  File "C:\Users\trrallele\AppData\Local\anaconda3\Lib\asyncio\base_events.py", line 651, in run_until_complete
    raise RuntimeError('Event loop stopped before Future completed.')
RuntimeError: Event loop stopped before Future completed.


📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
📈 ETH/USD Top of Book → Bid: 2393.09 | Ask: 2393.1
🛑 Stopping client...
